In [ ]:
%matplotlib inline
#%matplotlib notebook

For high dpi displays.

In [ ]:
%config InlineBackend.figure_format = 'retina'

# 0. General note

- This notebook fits a BM3 EoS and propagates uncertainties.
- Primarily uses burnman and LMFit packages.
- Loads in data from single LOG files.
- Loads in ruby spectrum .asc files.
- Fits ruby spectrum with 2 Lorentzians and a linear background.
- John Lazarz - 171221

# 1. General setup

In [ ]:
import os
import sys
import numpy as np
from scipy import constants as con
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import uncertainties as uct
from uncertainties import unumpy as unp
from lmfit import Model
from lmfit.models import LinearModel, LorentzianModel
import pandas as pd
from IPython.display import display
import burnman
import glob
import re

In [ ]:
if not os.path.exists('burnman') and os.path.exists('../burnman'):
    sys.path.insert(1, os.path.abspath('..'))

### What sample is this? - Where are the data folders?

In [ ]:
sample = 'Acetaminophen'
T = 298.15

ruby = r'/Users/jlazarz/Documents/data/xrd/acetaminophen/dac/acetaminophen/ruby'
log = r'/Users/jlazarz/Documents/data/xrd/acetaminophen/dac/acetaminophen'

old_ruby = r'/Users/jlazarz/Documents/data/xrd/acetaminophen/dac/acetaminophen/dac1-failed/ruby'
old_log = r'/Users/jlazarz/Documents/data/xrd/acetaminophen/dac/acetaminophen/dac1-failed'
old_ruby_folder = os.path.abspath(old_ruby)
old_log_folder = os.path.abspath(old_log)
    
if os.path.exists(os.path.abspath(ruby)):
    ruby_folder = os.path.abspath(ruby)
    log_folder = os.path.abspath(log)
    print('Using working directory.')

else:
    print('Directory does not exist.......')


# 2. Read data

Load in the .log files from directory as well as ruby .asc files from a folder named "ruby" within the directory.


In [ ]:
#idx_file = 0
#sample = logs[idx_file]
os.chdir(ruby_folder)
rubys = glob.glob('*.asc')
print('%s\n' % rubys)
print(sample)

# 3. Organize and fit data

First, we sort the ruby spectra into 2 lists. One list for 'table_rubys' and one list for 'sample_rubys'.

#### extract_ruby
In the 'extract_ruby' function we load the ruby spectrum from '.asc' files into lists. We then fit the data in the 'ruby_fit' funciton using the (Dawaele et al. 2008) equation to determine pressure. 

#### extract_cell
In the 'extract_cell' function we extract the cell information from the log file. This section searches through the log file and loads lattice parameters from the **last** "lsq" and the **last** "clsq" performed in Single in that log file. These lattice parameters, as well as standard deviations, are loaded into "cell" dictionary. 

Parameters collected from both "lsq" and "clsq": a, a_sigma, b, b_sigma, c, c_sigma, alpha, alpha_sigma, beta, beta_sigma ,gamma, gamma_sigma, volume, volume_sigma

To access parameters example:
 - To print "a" parameter from "lsq" function:
     - print(cell["lsq"]["a"])
 - To print "a_sigma" parameter from "clsq" function:
     - print(cell["clsq"]["a_sigma"])

In [ ]:
table_rubys = []
sample_rubys = []
for i in  rubys:
    if re.search('table',i):
        table_rubys.append(i)
    else:
        sample_rubys.append(i)

In [ ]:
def calc_pressure(sample_lam,table_lam):
    a = 1920.0
    b = 9.61
    r1 = sample_lam
    r1_o = table_lam
    p = (a/b)*(((r1/r1_o)**b)-1)
    return p

def normalize(y):
    z = (y-np.min(y))/(np.max(y)-np.min(y))
    return z

def ruby_fit(x,y_data,ruby):
    y = normalize(y_data)
    background  = LinearModel(prefix='lin_')
    pars = background.guess(y, x=x)

    r1 = LorentzianModel(prefix='r1_')
    pars.update(r1.make_params())
    pars['r1_center'].set(694.3)
    pars['r1_sigma'].set(0.000954)
    r2 = LorentzianModel(prefix='r2_')
    pars.update(r2.make_params())

    pars['r2_center'].set(692.9)
    pars['r2_sigma'].set(0.00135)
    mod = r1 + r2 + background
    init = mod.eval(pars, x=x)
    plt.plot(x, y)

    out = mod.fit(y, pars, x=x)

    comps = out.eval_components(x=x)

    print(out.fit_report(min_correl=0.5))

    plt.plot(x, out.best_fit, 'r-')
    plt.plot(x, comps['r1_'], 'b--')
    plt.plot(x, comps['r2_'], 'b--')
    plt.plot(x, comps['lin_'], 'k--')
    plt.title(ruby,fontweight='bold')
    plt.xlabel('Wavelength (nm)',fontweight='bold')
    plt.ylabel('Intensity (arb. units)',fontweight='bold')

    r1_center = str(out.params['r1_center']).split()[2]
    r1_center = float(r1_center.strip('value='))
    r1_esd = str(out.params['r1_center']).split()[4]
    r1_esd = float(r1_esd.strip(','))
    r2_center = str(out.params['r2_center']).split()[2]
    r2_center = float(r2_center.strip('value='))
    r2_esd = str(out.params['r2_center']).split()[4]
    r2_esd = float(r2_esd.strip(','))

    ruby_cent = [r1_center,r2_center]
    ruby_esd = [r1_esd,r2_esd]
    #print(ruby_cent)
    #print(ruby_esd)
    return ruby_cent

def extract_ruby(ruby):    
    with open(ruby, "r") as f:
        data = np.loadtxt(f)

    x = data[:,0]
    y = data[:,1]
    
    y_norm = normalize(y)
    return x,y_norm

def new_fit():
    #table ruby wavelength
    table_lam = []
    for i in table_rubys:
        fig = plt.figure()
        x,y = extract_ruby(i)
        r1,r2 = ruby_fit(x,y,i)
        table_lam.append(r1)

    sample_lam = []
    for i in sample_rubys:
        fig = plt.figure()
        x,y = extract_ruby(i)
        r1,r2 = ruby_fit(x,y,i)
        sample_lam.append(r1)

    dac_pressures = []
    for i in range(len(sample_lam)):
        p = calc_pressure(sample_lam[i],table_lam[i])
        dac_pressures.append(p)

    if dac_pressures[0] == 0:
        dac_pressures[0] = 1e-8

    print(dac_pressures)
    return dac_pressures

dac_pressures = new_fit()


In [ ]:
def old_fit():
    os.chdir(old_ruby_folder)
    rubys = glob.glob('*.asc')
    table_rubys = []
    sample_rubys = []
    for i in  rubys:
        if re.search('table',i):
            table_rubys.append(i)
        else:
            sample_rubys.append(i)
            
    #table ruby wavelength
    table_lam = []
    for i in table_rubys:
        fig = plt.figure()
        x,y = extract_ruby(i)
        r1,r2 = ruby_fit(x,y,i)
        table_lam.append(r1)

    sample_lam = []
    for i in sample_rubys:
        fig = plt.figure()
        x,y = extract_ruby(i)
        r1,r2 = ruby_fit(x,y,i)
        sample_lam.append(r1)

    dac_pressures = []
    for i in range(len(sample_lam)):
        p = calc_pressure(sample_lam[i],table_lam[i])
        dac_pressures.append(p)

    if dac_pressures[0] == 0:
        dac_pressures[0] = 1e-8

    print(dac_pressures)
    return dac_pressures

old_dac_pressures = old_fit()
os.chdir(ruby_folder)

In [ ]:
print('NEW PRESSURES: \n')
for i in range(len(dac_pressures)): 
    print('P%s : %.3f GPa' % (i,dac_pressures[i]))
    
print('\nOLD PRESSURES: \n')
for i in range(len(old_dac_pressures)): 
    print('P%s : %.3f GPa' % (i,old_dac_pressures[i]))

# 4. Retrieve Lattice Parameters

In [ ]:
def extract_cell(log):
    lsqstart = []
    clsqstart = []
    stop = []
    with open(log, "r") as f:
        searchlines = f.readlines()
    for i, line in enumerate(searchlines):
        if ">lsq" in line: 
            #for l in searchlines[i:i+3]: print(l)
            lsqstart.append(i)
        if ">clsq" in line:
            clsqstart.append(i)
        if "Sigmas " in line:
            stop.append(i)
    lsqend = []
    clsqend = []
    for i, spot in enumerate(stop):
        if spot > lsqstart[len(lsqstart)-1]:
            lsqend.append(spot)
        if spot > clsqstart[len(clsqstart)-1]:
            clsqend.append(spot)

    lsq_range = [lsqstart[len(lsqstart)-1],lsqend[0]]
    clsq_range = [clsqstart[len(clsqstart)-1],clsqend[0]]

    lsq_dat = searchlines[lsq_range[0]:lsq_range[1]+1]
    clsq_dat = searchlines[clsq_range[0]:clsq_range[1]+1]

    key = ['a','b','c','alpha','beta','gamma','vol',\
           'a_sigma','b_sigma','c_sigma',\
           'alpha_sigma','beta_sigma','gamma_sigma','vol_sigma']
    lsq_cell = lsq_dat[len(lsq_dat)-2].split()
    #the first element of each is the string 'CELL' so I delete it here. 
    del lsq_cell[0]
    lsq_cell = list(map(float,lsq_cell))
    lsq_sigma = lsq_dat[len(lsq_dat)-1].split()
    del lsq_sigma[0]
    lsq_sigma = list(map(float,lsq_sigma))
    lsq = lsq_cell+lsq_sigma
    clsq_cell = clsq_dat[len(clsq_dat)-2].split()
    del clsq_cell[0]
    clsq_cell = list(map(float,clsq_cell))
    clsq_sigma = clsq_dat[len(clsq_dat)-1].split()
    del clsq_sigma[0]
    clsq_sigma = list(map(float,clsq_sigma))
    clsq = clsq_cell+clsq_sigma

    lsq = dict(zip(key,lsq))
    clsq = dict(zip(key,clsq))
    cell = {"lsq":lsq,"clsq":clsq}
    return cell

def new_log():
    os.chdir(log_folder)
    logs = glob.glob('*.LOG')

    lattice_params = []
    for i in logs:
        cell = extract_cell(i)
        lattice_params.append(cell)
        #print(cell["lsq"]["a"])

    for i in range(len(lattice_params)):
        cell_pandas = pd.DataFrame(lattice_params[i])
        print('P%s - %.3f GPa' % (i,dac_pressures[i]))
        display(cell_pandas)
    return lattice_params

def old_log():
    os.chdir(old_log_folder)
    logs = glob.glob('*.LOG')

    lattice_params = []
    for i in logs:
        cell = extract_cell(i)
        lattice_params.append(cell)
        #print(cell["lsq"]["a"])

    for i in range(len(lattice_params)):
        cell_pandas = pd.DataFrame(lattice_params[i])
        print('P%s - %.3f GPa' % (i,old_dac_pressures[i]))
        display(cell_pandas)
    os.chdir(log_folder)
    logs = glob.glob('*.LOG')
    return lattice_params

lattice_params = new_log()
old_lattice_params = old_log()

# 5. Organize PV data into arrays.
Make a `numpy` array for volume and pressure data. [$P(GPa),\sigma_P,V({\unicode{x212B}}^3),\sigma_V$].

In [ ]:
def new_data():
    pv_lsq = np.zeros((len(lattice_params),4))
    pv_clsq = np.zeros((len(lattice_params),4))

    for i in range(len(lattice_params)):
        pv_lsq[i,0] = dac_pressures[i]
        pv_lsq[i,1] = 0.00001
        lp = lattice_params[i]
        pv_lsq[i,2] = lp["lsq"]["vol"]
        pv_lsq[i,3] = lp["lsq"]["vol_sigma"]

        pv_clsq[i,0] = dac_pressures[i]
        pv_clsq[i,1] = 0.0
        pv_clsq[i,2] = lp["clsq"]["vol"]
        pv_clsq[i,3] = lp["clsq"]["vol_sigma"]
    return pv_lsq,pv_clsq

def old_data():
    old_pv_lsq = np.zeros((len(old_lattice_params),4))
    old_pv_clsq = np.zeros((len(old_lattice_params),4))

    for i in range(len(old_lattice_params)):
        old_pv_lsq[i,0] = old_dac_pressures[i]
        old_pv_lsq[i,1] = 0.00001
        old_lp = old_lattice_params[i]
        old_pv_lsq[i,2] = old_lp["lsq"]["vol"]
        old_pv_lsq[i,3] = old_lp["lsq"]["vol_sigma"]

        old_pv_clsq[i,0] = old_dac_pressures[i]
        old_pv_clsq[i,1] = 0.0
        old_pv_clsq[i,2] = old_lp["clsq"]["vol"]
        old_pv_clsq[i,3] = old_lp["clsq"]["vol_sigma"]
    return old_pv_lsq,old_pv_clsq

pv_lsq,pv_clsq = new_data()
old_pv_lsq,old_pv_clsq = old_data()

# 6. Assign z (number of formula units per unit cell).

-- User edits this.

In [ ]:
z = 4.

v_data = np.delete(pv_clsq,[0,1],1)
print('Least squares equation of state fitting\n')

print('- Room temperature PV data\n')
print('clsq: \n')
print(pv_clsq)

print('\nlsq: \n')
print(pv_lsq)

# 7. Plot the data.

In [ ]:
#plt.errorbar(old_pv_lsq[:,0], old_pv_lsq[:,2], fmt='ro', \
#             xerr=old_pv_lsq[:,1], yerr=old_pv_lsq[:,3],label='old_lsq',fillstyle='none')
plt.errorbar(old_pv_clsq[:,0], old_pv_clsq[:,2], fmt='bo', \
             xerr=old_pv_clsq[:,1], yerr=old_pv_clsq[:,3],label='old_clsq')

#plt.errorbar(pv_lsq[:,0], pv_lsq[:,2], fmt='ko', \
#             xerr=pv_lsq[:,1], yerr=pv_lsq[:,3],label='lsq',fillstyle='none')
plt.errorbar(pv_clsq[:,0], pv_clsq[:,2], fmt='ko', \
             xerr=pv_clsq[:,1], yerr=pv_clsq[:,3],label='new_clsq')
plt.xlabel('Pressure (GPa)',fontweight='bold')
plt.ylabel('Volume ($\mathbf{\AA^3}$)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s EoS (%sK)" % (sample,T),fontweight='bold')
plt.legend()
#plt.savefig('PV-plot.png',dpi=800)
plt.show()
pv_lsq,pv_clsq = old_data()

### Note - the following actions are performed with "clsq" data.

### F-${f}$ plot

Normalised pressure : $F_E=\frac{P}{3f_E(1+2f_E)^{5/2}}$

$\sigma_F=F_E\sqrt{(\frac{\sigma_P}{P})^2+(\sigma')^2}$

Eulerian Finite-strain : $f_E=\frac{1}{2}\left[\left(\frac{V}{V_0}\right)^{-2/3}-1\right]$

$\sigma_f=\frac{1}{3}\eta^{-5/3}\sigma_\eta$

$\sigma'=\frac{(7\eta^{-2/3}-5)\sigma_\eta}{3(1-\eta^{-2/3})\eta}$

$\eta=\frac{V}{V_0}$

In [ ]:
p = pv_clsq[:,0]
sigP = pv_clsq[:,1]
V = pv_clsq[:,2]
sigV = pv_clsq[:,3]
Vo = pv_clsq[0,0]

sigVo = pv_clsq[0,1]
np.seterr(divide='ignore', invalid='ignore')
f = (1.0/2.0)*(((V/Vo)**(-2.0/3.0))-1.0)
F = p/(3.*f*(1.+(2.*f))**(5./2.))
eta = V/Vo
sigeta = np.abs(eta)*((((sigV/V)**2.0)+((sigVo/Vo)**2))**(1.0/2.0))
sigprime = ((7.0*(eta**(-2.0/3.0))-5.0)*sigeta)/(2.0*(1.0-(eta**-2.0/3.0))*eta)
sigF = F*np.sqrt(((sigP/p)**2.0)+(sigprime**2))

line_mod = LinearModel()
pars = line_mod.guess(np.delete(F,0), x=np.delete(f,0))
out = line_mod.fit(np.delete(F,0), pars, x=np.delete(f,0))
plt.plot(np.delete(f,0), out.best_fit, '-',color='black')

plt.errorbar(f, F, fmt='ko', xerr=0, yerr=sigF,\
             label=sample,alpha=1.0,capsize = 3.)
plt.xlabel('Eulerian strain $\mathit{f}$',fontweight='bold')
plt.ylabel('Normalised pressure $\mathbf{F_E}$ (GPa)',fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s $\mathit{f}$-F (%sK)" % (sample,T),fontweight='bold')
plt.legend()
#plt.savefig('Ff-plot.png',dpi=800)
plt.show()

# 8. Find initial starting parameters from Burnman dictionary.

In [ ]:
#smp = burnman.minerals.HP_2011_ds62.stv()
#smp = burnman.minerals.HP_2011_ds62.maj()
path = os.path.abspath(burnman.minerals.HP_2011_ds62.__file__)
print('Lookup file path: %s\n' % path)
smp = burnman.minerals.HP_2011_ds62.acet()
print(smp.params)

### Create array in correct units for burnman (P(Pa),T(K),V(m$^3$/mol))

In [ ]:
PTV = np.array([pv_clsq.T[0]*1.e9,
                pv_clsq.T[1]*0. + 298.15,
                burnman.tools.molar_volume_from_unit_cell_volume(pv_clsq.T[2], z)]).T
print(PTV)

### Create covariance matrix.

In [ ]:
nul = 0.*PTV.T[0]
PTV_covariances = np.array([[0.03*PTV.T[0], nul, nul], [nul, nul, nul], 
                            [nul, nul, burnman.tools.molar_volume_from_unit_cell_volume(pv_clsq.T[3], z)]]).T

# 9. Fit BM3 eos.

Birch-Murnaghan (isothermal): $P=3K_0f\left(1+2f\right)^{5/2}\left[1+\frac{3}{2}\left(K_0^\prime -4\right) f\right]$

In [ ]:
help(burnman.eos_fitting.fit_PTV_data)
params = ['V_0', 'K_0', 'Kprime_0']
fitted_eos = burnman.eos_fitting.fit_PTV_data(smp, params, PTV, PTV_covariances, verbose=True)


### Print optimized parameters.

In [ ]:
print('Optimized equation of state for %s:' % sample)
print('volume:[m^3/mol]\nK:[Pa]\n')
burnman.tools.pretty_print_values(fitted_eos.popt, fitted_eos.pcov, fitted_eos.fit_params)
print(fitted_eos.pcov)

### Create confidence ellipse plots.

In [ ]:
def plt_ellipse(cov, pos, nstdl, ax=None):
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()
    
    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    palet = ['red', 'cyan', 'blue']

    for i in range(1,4):
        nstd = nstdl[i-1]
        hue = palet[i-1]
        width, height = 2 * nstd * np.sqrt(vals)
        ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, color = hue, alpha=0.8)
        ax.add_artist(ellip)
        ax.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
        
    return ellip

In [ ]:
n_params = len(fitted_eos.pcov[0])
scaling = [(z/(con.N_A))*(1e30),1e-9,1e0]
scaling = np.outer(scaling,scaling)

fig, ax_array = plt.subplots(n_params-1, n_params-1)
nstd = [3.0,2.0,1.0]
err = [np.sqrt(fitted_eos.pcov[0,0])*(z/(con.N_A))*(1e30),np.sqrt(fitted_eos.pcov[1,1])*1e-9,\
        np.sqrt(fitted_eos.pcov[2,2])]
err = np.outer(err,err)


for i in range(n_params-1):
    
    for j in range(i+1, n_params):
        indices = np.array([i, j])
        projected_cov = (fitted_eos.pcov*scaling)[indices[:, None], indices]
        
        scaled_pos = np.array([fitted_eos.popt[i]*np.sqrt(scaling[i][i]),
                               fitted_eos.popt[j]*np.sqrt(scaling[j][j])]) 
        
        cov = projected_cov
        pos = scaled_pos
            
        ellipse = plt_ellipse(cov,pos,nstd,ax=ax_array[j-1][i])
        maxx = 1.5*2.2*np.sqrt(projected_cov[0][0])
        maxy = 1.5*2.2*np.sqrt(projected_cov[1][1])
        ax_array[j-1][i].set_xlim(scaled_pos[0]-maxx, scaled_pos[0]+maxx)
        ax_array[j-1][i].set_ylim(scaled_pos[1]-maxy, scaled_pos[1]+maxy)
        ax_array[j-1][i].yaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))
        
        if i == 0 and j == 1:
            ax_array[j-1][i].errorbar(pos[0],pos[1], xerr=np.sqrt(err[0,0]), yerr=np.sqrt(err[1,1]),\
                    linestyle='None', marker='None', label=sample,color='black', capsize = 3.)
            #ax_array[j-1][i].errorbar(shinmei_vo[0],shinmei_ko[0], xerr=shinmei_vo[1], yerr=shinmei_ko[1],\
            #        linestyle='None', marker='^', label='Shinmei et al. (1999)', color='black', capsize = 3.)
            #ax_array[j-1][i].errorbar(angel_vo[0],angel_ko[0], xerr=angel_vo[1], yerr=angel_ko[1],\
            #        linestyle='None', marker='o', label='Angel and Hugh-Jones (1994)',color='black', capsize = 3.)
            print('V0 [A^3]: %s +/- %s' % (pos[0],np.sqrt(err[0,0])))
            v0 = uct.ufloat(pos[0], np.sqrt(err[0,0]))
            print('K0 [GPa]: %s +/- %s' % (pos[1],np.sqrt(err[1,1])))
        if i == 0 and j == 2:
            ax_array[j-1][i].errorbar(pos[0],pos[1], xerr=np.sqrt(err[0,0]), yerr=np.sqrt(err[2,2]),\
                    linestyle='None', marker='None', label=sample,color='black', capsize = 3.)
            #ax_array[j-1][i].errorbar(shinmei_vo[0],shinmei_kp[0], xerr=shinmei_vo[1], yerr=shinmei_kp[1],\
            #        linestyle='None', marker='^', label='Shinmei et al. (1999)', color='black', capsize = 3.)
            #ax_array[j-1][i].errorbar(angel_vo[0],angel_kp[0], xerr=angel_vo[1], yerr=0.0,\
            #        linestyle='None', marker='o', label='Angel and Hugh-Jones (1994)',color='black', capsize = 3.)
        if i == 1 and j == 2:
            ax_array[j-1][i].errorbar(pos[0],pos[1], xerr=np.sqrt(err[1,1]), yerr=np.sqrt(err[2,2]),\
                    linestyle='None', marker='None', label=sample,color='black', capsize = 3.)
            #ax_array[j-1][i].errorbar(shinmei_ko[0],shinmei_kp[0], xerr=shinmei_ko[1], yerr=shinmei_kp[1],\
            #        linestyle='None', marker='^', label='Shinmei et al. (1999)', color='black', capsize = 3.)
            #ax_array[j-1][i].errorbar(angel_ko[0],angel_kp[0], xerr=angel_ko[1], yerr=0.0,\
            #        linestyle='None', marker='o', label='Angel and Hugh-Jones (1994)',color='black', capsize = 3.)
            print('K0_prime: %s +/- %s' % (pos[1],np.sqrt(err[2,2])))
        
red_patch = mpatches.Patch(color='red', alpha=0.8, label='3$\sigma$')
cyan_patch = mpatches.Patch(color='cyan', alpha=0.8, label='2$\sigma$')
blue_patch = mpatches.Patch(color='blue', alpha=0.8, label='1$\sigma$')
us = mlines.Line2D([],[],linestyle='none',marker='+', color='black', label='This study')
#shinmei = mlines.Line2D([],[],linestyle='none',marker='^', color='black', label='Shinmei et al. (1999)')
#ross = mlines.Line2D([],[],linestyle='none',marker='o', color='black', label='Angel and Hugh-Jones (1994)')

ax_array[0][1].set_axis_off()
legend = ax_array[0][1].legend(handles=[red_patch,cyan_patch,blue_patch,us], loc = 'center')
legend.get_frame().set_linewidth(0.0)
param_names = ['$\mathbf{V_0}$','$\mathbf{K_0}$','$\mathbf{K_0}$\'']
param_units = ['$\mathbf{\AA^3}$','GPa','']
if param_names != []:
    for i in range(n_params-1):
        ax_array[n_params-2][i].set_xlabel('{0:s} ({1:s})'.format(param_names[i],param_units[i]),fontweight='bold')
    for j in range(1, n_params):
        if param_units[j] == param_units[2]:
            ax_array[j-1][0].set_ylabel('{0:s}'.format(param_names[j]),fontweight='bold')
        else:
            ax_array[j-1][0].set_ylabel('{0:s} ({1:s})'.format(param_names[j],param_units[j]),fontweight='bold')
        ax_array[j-1][0].yaxis.set_label_coords(-0.25,0.5)

#fig.savefig('confidence-ellipse.png',dpi=800)
plt.show()

# 10. Plot data and new EoS.

In [ ]:
maxp = np.amax(pv_clsq[:,0])*1.e9+(2.e9)
pressures = np.linspace(1.e5, maxp, 101)
volumes = np.empty_like(pressures)

PTVs = np.empty((len(pressures), 3))
for i, P in enumerate(pressures):
    smp.set_state(P, T)
    PTVs[i] = [P, T, smp.V]

# Plot the 95% confidence and prediction bands 
cp_bands = burnman.nonlinear_fitting.confidence_prediction_bands(model = fitted_eos, x_array = PTVs,
                                                                 confidence_interval = 0.95,
                                                                 f=burnman.tools.attribute_function(smp, 'V'),
                                                                 flag='V')

plt.plot(PTVs[:,0]/1.e9, cp_bands[0] * (z/(con.N_A))*(1e30), linestyle='--', linewidth=0.75, color='r', label='95% confidence bands')
plt.plot(PTVs[:,0]/1.e9, cp_bands[1] * (z/(con.N_A))*(1e30), linestyle='--', linewidth=0.75, color='r')


plt.plot(PTVs[:,0] / 1.e9, PTVs[:,2] * (z/(con.N_A))*(1e30), label='BM3 %s' % 'old_data')


#clino1 = np.delete(PTV, (6,7,8,9,10,11), axis=0)

#cov11 = np.delete(PTV_covariances.T[0][0],(6,7,8,9,10,11))
#cov21 = np.delete(PTV_covariances.T[2][2],(6,7,8,9,10,11))
#cov12 = np.delete(PTV_covariances.T[0][0],(0,1,2,3,4,5))
#cov22 = np.delete(PTV_covariances.T[2][2],(0,1,2,3,4,5))
cov11 = PTV_covariances.T[0][0]
cov21 = PTV_covariances.T[2][2]
cov12 = PTV_covariances.T[0][0]
cov22 = PTV_covariances.T[2][2]

plt.errorbar(PTV[:,0] / 1.e9, PTV[:,2] * (z/(con.N_A))*(1e30), xerr=cov11 / 1.e9, 
             yerr=cov21 * (z/(con.N_A))*(1e30),
             linestyle='None', marker='o', label='old_data',color='blue', alpha=1.0)

######################
pv_lsq,pv_clsq = new_data()
plt.errorbar(pv_clsq[:,0], pv_clsq[:,2], fmt='ko', \
             xerr=pv_clsq[:,1], yerr=pv_clsq[:,3],label='new_data')
######################

#plt.xlim([2.5,37.5])
#plt.ylim([325,400])
plt.ylabel("Volume ($\mathbf{\AA^3}$)",fontweight='bold')
plt.xlabel("Pressure (GPa)",fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.legend(loc="upper right")
plt.title("%s EoS (%sK)" % (sample,T),fontweight='bold')
#plt.savefig('PV_eos-plot-to9GPa.png',dpi=800)
plt.show()

### Plot bulk modulus.

Isothermal bulk modulus: 
$K_{T}=(1+2f)^{5/2}\biggl[  K_0+(3K_0{K}^\prime_{0}-5K_0)f+\frac{27}{2}(K_0{K}^\prime_{0}-4K_0)f^2 \biggr]$

Shear modulus: $G = (1+2f)^{5/2}  \biggl[G_0+(3K_0{G}^\prime_{0}-5G_0)f+(6K_0{G}^\prime_{0}-24K_0-14G_{0}
     + \frac{9}{2}K_{0}{K}^\prime_{0})f^2 \biggr]$

In [ ]:
cp_bands = burnman.nonlinear_fitting.confidence_prediction_bands(model = fitted_eos, x_array = PTVs,
                                                                 confidence_interval = 0.95,
                                                                 f=burnman.tools.attribute_function(smp, 'K_T'),
                                                                 flag='V')
array = np.array([[6.37389300e+09,2.98150000e+02,2.89943113e-05]])
plt.plot(PTVs[:,0]/1.e9, (cp_bands[0] + cp_bands[1])/2.e9, color='b', label='Best fit')
plt.plot(PTVs[:,0]/1.e9, (cp_bands[0])/1.e9, linestyle='--', linewidth=0.75, color='r', label='95% confidence band')
plt.plot(PTVs[:,0]/1.e9, (cp_bands[1])/1.e9, linestyle='--', linewidth=0.75, color='r')
cp_bands2 = burnman.nonlinear_fitting.confidence_prediction_bands(model = fitted_eos, x_array = array,
                                                                 confidence_interval = 0.95,
                                                                 f=burnman.tools.attribute_function(smp, 'K_T'),
                                                                 flag='V')
scale = (z/(con.N_A))*(1e30)
#print(scale*array[0,2])
#print((cp_bands2[0] + cp_bands2[1])/2.e9)
plt.ylabel("K (GPa)",fontweight='bold')
plt.xlabel("Pressure (GPa)",fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.legend(loc="upper left")
plt.title("%s Bulk Modulus (%sK)" % (sample,T),fontweight='bold')
#plt.savefig('PK-plot.png',dpi=800)
plt.show()

### Miscellaneous plots.

In [ ]:
a_len = []
b_len = []
c_len = []
a_sigma = []
b_sigma = []
c_sigma = []
beta_angle = []
beta_sigma = []

if len(dac_pressures) != len(lattice_params):
    x = len(dac_pressures)-len(lattice_params)
    del dac_pressures[(len(dac_pressures)-x):]
    
for i in range(len(lattice_params)):
    n_cell = lattice_params[i]
    a_len.append(n_cell["clsq"]["a"])
    b_len.append(n_cell["clsq"]["b"])
    c_len.append(n_cell["clsq"]["c"])
    a_sigma.append(n_cell["clsq"]["a_sigma"])
    b_sigma.append(n_cell["clsq"]["b_sigma"])
    c_sigma.append(n_cell["clsq"]["c_sigma"])
    beta_angle.append(n_cell["clsq"]["beta"])
    beta_sigma.append(n_cell["clsq"]["beta_sigma"])

print('a reduction: %.3f%%' % ((1-(a_len[len(a_len)-1]/a_len[0]))*100))
print('b reduction: %.3f%%' % ((1-(b_len[len(b_len)-1]/b_len[0]))*100))
print('c reduction: %.3f%%\n' % ((1-(c_len[len(c_len)-1]/c_len[0]))*100))                         

pars = line_mod.guess(a_len, x=dac_pressures)
out = line_mod.fit(a_len, pars, x=dac_pressures)
plt.plot(dac_pressures, out.best_fit, '-',color='black',linestyle='--', linewidth=0.75)
plt.errorbar(dac_pressures, a_len,yerr=a_sigma, color='red',marker='o',linestyle='none',
         label='a')
print('a: %s' % out.fit_report())

pars = line_mod.guess(b_len, x=dac_pressures)
out = line_mod.fit(b_len, pars, x=dac_pressures)
plt.plot(dac_pressures, out.best_fit, '-',color='black',linestyle='--', linewidth=0.75)
plt.errorbar(dac_pressures, b_len,yerr=b_sigma,color='blue',marker='o',linestyle='none',
        label='b')
print('b: %s' % out.fit_report())

pars = line_mod.guess(c_len, x=dac_pressures)
out = line_mod.fit(c_len, pars, x=dac_pressures)
plt.plot(dac_pressures, out.best_fit, '-',color='black',linestyle='--', linewidth=0.75)
plt.errorbar(dac_pressures, c_len,yerr=c_sigma,color='black',marker='o',linestyle='none',
        label='c')
print('c: %s' % out.fit_report())

plt.ylim([5.0,14.0])
plt.ylabel("Length ($\mathbf{\AA}$)",fontweight='bold')
plt.xlabel("Pressure (GPa)",fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.legend(loc="upper right")
plt.title("%s Lattice Parameters ($\mathbf{\AA}$)" % (sample),fontweight='bold')
#plt.savefig('lattice_parameters-plot.png',dpi=800)
plt.show()

In [ ]:
print('beta_angle increase: %.3f%%' % ((1-(beta_angle[0]/beta_angle[len(beta_angle)-1]))*100))  

plt.errorbar(dac_pressures, beta_angle, yerr = beta_sigma, color='black',marker='o',linestyle='none')
pars = line_mod.guess(beta_angle, x=dac_pressures)
out = line_mod.fit(beta_angle, pars, x=dac_pressures)
plt.plot(dac_pressures, out.best_fit, '-',color='black',linestyle='--', linewidth=0.75)

plt.ylabel("Angle (degrees)",fontweight='bold')
plt.xlabel("Pressure (GPa)",fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Beta Angle (degrees)" % (sample),fontweight='bold')
#plt.savefig('angle-plot.png',dpi=800)
plt.show()

### Density.

In [ ]:
print('V_0 [A^3]: %s' % v0)
#Acetaminophen = C8H9NO2
mass = (((12.011*8)+(1.008*9)+(14.007)+(15.999*2))/(con.N_A))*z
vol_cm3 = v0*(1e-24)
den = mass/vol_cm3
print('Density_0 [g/cm^3]: %s' % den)

volumes = (PTVs[:,2] * (z/(con.N_A))*(1e30))*(1e-24)
densities = mass/volumes

obs_densities = mass/(pv_clsq[:,2]*(1e-24))

plt.plot((PTVs[:,0]*1e-9), densities, '-',color='black',linestyle='-', linewidth=1.0)
plt.plot(dac_pressures,obs_densities,'ko')
plt.ylabel("Density $\mathbf{(g/cm^3)}$",fontweight='bold')
plt.xlabel("Pressure (GPa)",fontweight='bold')
plt.tick_params(direction='in',bottom=1,top=1,left=1,right=1)
plt.title("%s Density" % (sample),fontweight='bold')
#plt.savefig('density-plot.png',dpi=800)
plt.show()